![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FGenerate&file=Long+Context+Retrieval+With+The+Vertex+AI+Gemini+API.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGenerate%2FLong%2520Context%2520Retrieval%2520With%2520The%2520Vertex%2520AI%2520Gemini%2520API.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Long Context Retrieval With The Vertex AI Gemini API

**Retrieval - the task of retrieving information as context for an LLM**, like the Gemini family on Vertex AI. Retrieval augmented generation (RAG) is the task of retrieving relevant context and then providing it along with the prompt to the LLM. 

[Long context](https://cloud.google.com/vertex-ai/generative-ai/docs/long-context) is a way of providing full-length sources to the LLM, which can then perform its own retrieval.  Gemini 1.5 Flash (1M) and Gemini 1.5 Pro (2M) have incredible input context windows (1M and 2M tokens respectively) and have shown [near-perfect retrieval of > 99%](https://storage.googleapis.com/deepmind-media/gemini/gemini_v1_5_report.pdf).

For a complete overview of the Gemini API, check out the companion workflow [Vertex AI Gemini API](./Vertex%20AI%20Gemini%20API.ipynb).

**Use Case Exploration**

Buying a home usually involves borrowing money from a lending institution, typically through a mortgage secured by the home's value. But how do these institutions manage the risks associated with such large loans, and how are lending standards established?

In the United States, two government-sponsored enterprises (GSEs) play a vital role in the housing market:
- Federal National Mortgage Association ([Fannie Mae](https://www.fanniemae.com/))
- Federal Home Loan Mortgage Corporation ([Freddie Mac](https://www.freddiemac.com/))

These GSEs purchase mortgages from lenders, enabling those lenders to offer more loans. This process also allows Fannie Mae and Freddie Mac to set standards for mortgages, ensuring they are responsible and borrowers are more likely to repay them. This system makes homeownership more affordable and stabilizes the housing market by maintaining a steady flow of liquidity for lenders and keeping interest rates controlled.

However, navigating the complexities of these GSEs and their extensive servicing guides can be challenging. What if you could directly query these guides and get precise answers without needing to design a complex RAG architecture?

This workflow leverages the long context capabilities of Vertex AI Gemini models and the efficiency of context caching to provide low-latency and cost-effective access to these comprehensive documents. Explore the implementation below!

**References**
- [Retrieval Augmented Generation or Long-Context LLMs? A Comprehensive Study and Hybrid Approach](https://arxiv.org/pdf/2407.16833)


---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('fitz', 'pymupdf'),
    ('requests', 'requests')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com



To take a quick anonymous survey, run:
  $ gcloud survey



### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'long-context'

GCS_BUCKET = PROJECT_ID # change to Bucket name if not the same as the Project ID

packages:

In [8]:
# Python standard library imports:
import os, io, base64, json, datetime

# package imports
from IPython.display import Markdown
import IPython.display
import fitz #pymupdf
import requests

# vertex ai imports
from google.cloud import aiplatform
import vertexai
import vertexai.generative_models # for Gemini Models

# preview imports for vertex ai api:
from vertexai.preview import caching
import vertexai.preview.generative_models
import vertexai.preview.batch_prediction

# google cloud imports
from google.cloud import storage

In [9]:
aiplatform.__version__

'1.71.0'

clients:

In [10]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

---
## Gemini Models

Select one of the [supported Gemini models](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported-models) and read more about the characteristics of each [here](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


### Setup Model

Here the [Gemini 1.5 Flash model with version 002](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-1.5-flash) is selected. It has these characteristics (to name a few):
- Max Input Tokens: 1,048,576
- Max Output Tokens: 8,192
- Max image:
    - raw size 20MB
    - base64 encoded size 7MB
    - number per prompt 3000
- Max video:
    - length 1 hour
    - number per prompt 10
- Max audio:
    - length 8.4 hours
    - number per prompt 1
- Max PDF:
    - size 30 MB
- 102 [Languages](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#languages-gemini) for understanding and responding


In [11]:
gemini = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")

### **Prompt With Text**

In [12]:
response = gemini.generate_content('How do I get a mortgage?')
Markdown(response.text)

Getting a mortgage involves several steps and requires careful planning. Here's a general overview of the process:

**1. Check your credit score and report:**

* **Obtain your credit report:** You're entitled to a free credit report annually from each of the three major credit bureaus (Equifax, Experian, and TransUnion) through AnnualCreditReport.com.  Review it for errors and address any issues that could negatively impact your score.
* **Understand your credit score:** Your credit score is a crucial factor in mortgage approval and interest rates. Higher scores typically qualify you for better terms.  Aim for a score of 620 or higher, although some lenders may accept lower scores with compensating factors.

**2. Determine how much you can afford:**

* **Use online mortgage calculators:** These tools can help you estimate your monthly payments based on various loan amounts, interest rates, and loan terms.
* **Consider your debt-to-income ratio (DTI):** Lenders use DTI to assess your ability to repay the loan.  A DTI of 43% or lower is generally preferred, but some lenders may accept higher ratios. Calculate your DTI by dividing your total monthly debt payments by your gross monthly income.
* **Factor in additional costs:** Remember to account for property taxes, homeowners insurance, private mortgage insurance (PMI if your down payment is less than 20%), closing costs, and potential moving expenses.

**3. Get pre-approved for a mortgage:**

* **Shop around with multiple lenders:** Compare interest rates, fees, and loan terms from different lenders, including banks, credit unions, and online mortgage companies.
* **Provide necessary documentation:** Lenders will require documentation such as pay stubs, tax returns, bank statements, and employment history.
* **Receive a pre-approval letter:** This letter indicates how much the lender is willing to loan you and strengthens your position when making an offer on a home.

**4. Shop for a home:**

* **Work with a real estate agent:** A qualified agent can help you find properties that meet your needs and budget.
* **Make an offer:** Once you've found a home you like, submit a written offer to the seller.
* **Negotiate the terms:** Be prepared to negotiate the price, closing date, and other contingencies.

**5. Finalize the mortgage:**

* **Complete the loan application:** Provide the lender with all required documentation.
* **Get a home appraisal:** The lender will order an appraisal to determine the fair market value of the property.
* **Review the closing disclosure:** This document outlines the final loan terms, including closing costs.
* **Close on the loan:** Sign the final paperwork and pay closing costs.

**Tips for a smooth mortgage process:**

* **Save for a down payment:** A larger down payment can lead to lower interest rates and eliminate the need for PMI.
* **Pay down debt:** Reducing your debt can improve your DTI and credit score.
* **Avoid making major financial changes:** Don't open new credit accounts, change jobs, or make large purchases during the mortgage process.
* **Be organized and responsive:** Keep all your financial documents readily available and respond to lender requests promptly.
* **Ask questions:** Don't hesitate to ask your lender or real estate agent if you have any questions or concerns.


Getting a mortgage can be complex, but by following these steps and being prepared, you can navigate the process successfully. Remember to shop around and compare offers from multiple lenders to ensure you're getting the best possible terms.


### Retrieve Documents

In [13]:
freddie_url = 'https://guide.freddiemac.com/ci/okcsFattach/get/1002095_2'
fannie_url = 'https://singlefamily.fanniemae.com/media/39861/display'

In [14]:
freddie_retrieve = requests.get(freddie_url).content
fannie_retrieve = requests.get(fannie_url).content

In [15]:
freddie_doc = fitz.open(stream = freddie_retrieve, filetype = 'pdf')
fannie_doc = fitz.open(stream = fannie_retrieve, filetype = 'pdf')

In [16]:
freddie_doc.page_count, fannie_doc.page_count

(2588, 1180)

### Split Documents

The models have constraints on the size of individual files.  Here we want to split the PDFs into parts of no more than 1000 pages and verify that they are under 30MB in size.

In [17]:
def doc_parts(doc):
    start_page = 0
    max_pages = 1000
    n_pages = doc.page_count
    
    doc_list = []
    while start_page < n_pages:
        end_page = min(start_page + max_pages - 1, n_pages)
        new_doc = fitz.open()
        new_doc.insert_pdf(doc, from_page = start_page, to_page = end_page)
        doc_list.append(new_doc)
        start_page = end_page + 1
    
    print(f"The document has {n_pages} pages and has been split into parts with page counts: {[p.page_count for p in doc_list]}")
    
    return doc_list

In [18]:
freddie_parts = doc_parts(freddie_doc)

The document has 2588 pages and has been split into parts with page counts: [1000, 1000, 588]


In [19]:
fannie_parts = doc_parts(fannie_doc)

The document has 1180 pages and has been split into parts with page counts: [1000, 180]


### Save Documents To GCS Files

In [20]:
def doc_to_gcs(document, name):
    buffer = io.BytesIO()
    document.save(buffer)
    buffer.seek(0) # reset the position to the beginning
    blob = bucket.blob(f"{SERIES}/{EXPERIMENT}/{name}.pdf")
    blob.upload_from_file(buffer, content_type = 'application/pdf')
    print(f"The file 'gs://{bucket.name}/{blob.name}' is {(blob.size / (1024*1024)):.2f} MB")
    return blob

In [21]:
freddie_blob = doc_to_gcs(freddie_doc, 'freddie_full')

The file 'gs://statmike-mlops-349915/applied-genai/long-context/freddie_full.pdf' is 14.60 MB


In [22]:
fannie_blob = doc_to_gcs(fannie_doc, 'fannie_full')

The file 'gs://statmike-mlops-349915/applied-genai/long-context/fannie_full.pdf' is 4.55 MB


### Save Document Parts To GCS Files

In [23]:
freddie_blobs = [doc_to_gcs(doc, f'freddie_part_{d}') for d, doc in enumerate(freddie_parts)]

The file 'gs://statmike-mlops-349915/applied-genai/long-context/freddie_part_0.pdf' is 4.79 MB
The file 'gs://statmike-mlops-349915/applied-genai/long-context/freddie_part_1.pdf' is 5.21 MB
The file 'gs://statmike-mlops-349915/applied-genai/long-context/freddie_part_2.pdf' is 4.28 MB


In [24]:
fannie_blobs = [doc_to_gcs(doc, f'fannie_part_{d}') for d, doc in enumerate(fannie_parts)]

The file 'gs://statmike-mlops-349915/applied-genai/long-context/fannie_part_0.pdf' is 3.49 MB
The file 'gs://statmike-mlops-349915/applied-genai/long-context/fannie_part_1.pdf' is 0.61 MB


### Gemini Multimodal Context Parts

In [25]:
freddie_contexts = [
    vertexai.generative_models.Part.from_uri(
        uri = f"gs://{bucket.name}/{b.name}",
        mime_type = b.content_type
    ) for b in freddie_blobs
]

In [26]:
fannie_contexts = [
    vertexai.generative_models.Part.from_uri(
        uri = f"gs://{bucket.name}/{b.name}",
        mime_type = b.content_type
    ) for b in fannie_blobs
]

---
## Multimodal Prompts With Long Context

### Generate Responses With Gemini Multimodal Prompts - Long Context In Prompt

Use the Gemini Flash 1.5 model separately for each of the three prompt example: specific to Fannie Mae, Freddie Mac, and then a combined and comparative answer.

In [29]:
prompt = 'Does a lender have to perform servicing functions directly?'

In [30]:
freddie_response = gemini.generate_content(['The Freddie Mac documents:'] + freddie_contexts + [prompt])
Markdown(freddie_response.text)

Lenders are not required to perform servicing functions directly. They can contract with a third-party servicing company. However, in some situations, they might find it advantageous to maintain that role in-house, especially if the loan requires special handling, such as in cases of loss mitigation.

In [31]:
fannie_response = gemini.generate_content(['The Fannie Mae documents:'] + fannie_contexts + [prompt])
Markdown(fannie_response.text)

No. Lenders may outsource some or all of their servicing functions, but they are still fully responsible to Fannie Mae for those functions. This includes the use of a subservicer or service bureau. If a lender uses a subservicer, then both the lender (as the "master servicer") and the subservicer must be Fannie Mae-approved servicers in good standing and able to perform the duties associated with a master servicer/subservicer arrangement.

Additionally, a lender may use a document custodian to hold and manage certain loan documents on behalf of Fannie Mae and the lender. Fannie Mae has approved a list of document custodians and lenders are required to choose from this list.

In [32]:
gemini_gse = vertexai.generative_models.GenerativeModel(
    model_name = "gemini-1.5-pro-002",
    system_instruction = 'You are incredibly knowledgable about GSEs (Freddie Mac and Fannie Mae) who purchase mortgages from lenders.  You answer questions about the selling process from the point of view of each GSE and then you compare/contrast each of them relative to the users question.'
)
combined_response = gemini_gse.generate_content(['The Freddie Mac documents:'] + freddie_contexts + ['The Fannie Mae documents:'] + fannie_contexts + [prompt])
Markdown(combined_response.text)

**Freddie Mac's Perspective**

Freddie Mac allows lenders to transfer their servicing responsibilities to another Freddie Mac-approved servicer through a variety of methods, including Concurrent Transfer of Servicing, Subsequent Transfer of Servicing, and Intra-Servicer Portfolio Moves.  Freddie Mac monitors both the transferor and transferee servicer for compliance and has the right to rescind or deny servicing transfers in certain situations.


**Fannie Mae's Perspective**

Fannie Mae requires a lender to transfer the servicing rights to another Fannie Mae-approved servicer if it cannot or does not intend to service the loans itself.  Concurrent Transfers of Servicing occur when a lender sells loans to Fannie Mae. Subsequent Transfers of Servicing occur after Fannie Mae has purchased or securitized a loan.  Similar to Freddie Mac, Fannie Mae may monitor and assess the transferor and transferee servicers.

**Comparison**

Both GSEs require that loans be serviced by an approved servicer, allowing lenders to transfer these responsibilities if needed.  They both monitor the process for compliance and retain the right to step in under certain circumstances.  The specific processes and requirements for transferring servicing may vary slightly between the two GSEs.


---
## Multimodal Prompts With Context Cache For Long Context

### Create Context Cache(s)

Rather than sending the document (parts) along with each call to the Gemini API, it can be helpful to first load the documents as a context cache.  This makes subsequent call to the API faster and possibly cheaper as the documents are charged at a caching rate (size and time used) rather than a token/character rate.  

For more information on [Context Caching](https://cloud.google.com/vertex-ai/generative-ai/docs/context-cache/context-cache-overview) check out the companion workflow: [Vertex AI Gemini API](./Vertex%20AI%20Gemini%20API.ipynb).

In [34]:
freddie_cache = vertexai.preview.caching.CachedContent.create(
    model_name = 'gemini-1.5-flash-002',
    contents = freddie_contexts,
    ttl = datetime.timedelta(minutes = 30),
    display_name = 'freddie-cache'
)

In [35]:
fannie_cache = vertexai.preview.caching.CachedContent.create(
    model_name = 'gemini-1.5-flash-002',
    contents = fannie_contexts,
    ttl = datetime.timedelta(minutes = 30),
    display_name = 'fannie-cache'
)

In [36]:
combined_cache = vertexai.preview.caching.CachedContent.create(
    model_name = 'gemini-1.5-flash-002',
    contents = ['The Freddie Mac documents:'] + freddie_contexts + ['The Fannie Mae documents:'] + fannie_contexts,
    system_instruction = 'You are incredibly knowledgable about GSEs (Freddie Mac and Fannie Mae) who purchase mortgages from lenders.  You answer questions about the selling process from the point of view of each GSE and then you compare/contrast each of them relative to the users question.',
    ttl = datetime.timedelta(minutes = 30),
    display_name = 'combined-cache'
)

### Generate Responses With Gemini Multimodal Prompts - Using Cache

Register the Gemini Flash 1.5 model separately for each of the three caches. Then prompt each to see answer specific to Fannie Mae, Freddie Mac, and then a combined and comparative answer.

In [37]:
prompt = 'Does a lender have to perform servicing functions directly?'

In [38]:
freddie_model = vertexai.preview.generative_models.GenerativeModel.from_cached_content(cached_content = freddie_cache)
fannie_model = vertexai.preview.generative_models.GenerativeModel.from_cached_content(cached_content = fannie_cache)
combined_model = vertexai.preview.generative_models.GenerativeModel.from_cached_content(cached_content = combined_cache)

In [39]:
freddie_response = freddie_model.generate_content(
    contents = [prompt]
)
Markdown(freddie_response.text)

No, a lender does not have to perform servicing functions directly.  The Freddie Mac Single-Family Seller/Servicer Guide outlines situations where lenders may use a Servicer to perform those functions.  The guide also details the responsibilities and requirements for both the lender and the servicer in such arrangements.


In [40]:
fannie_response = fannie_model.generate_content(
    contents = [prompt]
)
Markdown(fannie_response.text)

No.  A lender may use other organizations to perform some or all of its servicing functions.  Fannie Mae refers to these arrangements as "subservicing" arrangements.  A master servicer may use a subservicer, but the master servicer remains ultimately responsible for meeting Fannie Mae's requirements.

In [41]:
combined_response = combined_model.generate_content(
    contents = [prompt]
)
Markdown(combined_response.text)

Here's a comparison of Freddie Mac and Fannie Mae's perspectives on whether a lender must perform servicing functions directly, based on their Selling Guides:

**Freddie Mac:**

Freddie Mac's Selling Guide doesn't require lenders to perform servicing functions directly.  The Guide explicitly allows for servicing to be transferred to a third-party servicer.  There are extensive sections dedicated to Transfers of Servicing, outlining the requirements and responsibilities of both the transferring servicer and the transferee servicer.  The process involves obtaining Freddie Mac's approval and adhering to specific timelines and procedures.

**Fannie Mae:**

Similar to Freddie Mac, Fannie Mae's Selling Guide also permits servicing to be performed by a third party.  The Guide details requirements for transfers of servicing, including the need for Fannie Mae's approval and various documentation requirements.  There's a strong emphasis on ensuring the transferee servicer meets Fannie Mae's standards and can adequately service the mortgages.

**Comparison:**

Both Freddie Mac and Fannie Mae allow for third-party servicing. Their Selling Guides provide detailed processes and requirements for transferring servicing rights, emphasizing the need for approval and adherence to specified procedures.  There are no explicit requirements mandating that lenders service the loans themselves.  The differences in specific requirements between the two GSEs are primarily in their detailed processes and the specific forms and procedures.  Both strive to ensure the mortgages are serviced according to their standards, regardless of who performs the servicing.


### Check And Remove The Context Cache(s)

Check the remaining time for each cache.  This time can be extended as needed with `.update()`.  In this case the caches are deleted to eliminate any further costs now that this workflow is complete.

In [42]:
def time_left(cache):
    expire = cache.expire_time
    now = datetime.datetime.now(tz=expire.tzinfo) 
    print(f"Expiration Time: {expire.strftime('%B %d, %Y at %I:%M:%S %p')}")
    print(f"   Current Time: {now.strftime('%B %d, %Y at %I:%M:%S %p')}")
    diff = (expire - now).total_seconds()
    sign, diff = (1, abs(diff)) if diff >= 0 else (-1, abs(diff))
    minutes = int(diff // 60)
    seconds = int(diff % 60)
    if minutes > 60:
        hours = int(minutes // 60)
        minutes = minutes - hours*60
    else: hours = 0
    if sign == 1:
        print(f"{hours:02d}:{minutes:02d}:{seconds:02d} until expiration")
    else:
        print(f"Expired {hours:02d}:{minutes:02d}:{seconds:02d} ago")
    return

In [43]:
time_left(freddie_cache)

Expiration Time: January 26, 2025 at 03:28:28 PM
   Current Time: January 26, 2025 at 03:19:49 PM
00:08:38 until expiration


In [44]:
time_left(fannie_cache)

Expiration Time: January 26, 2025 at 03:40:27 PM
   Current Time: January 26, 2025 at 03:19:49 PM
00:20:37 until expiration


In [45]:
time_left(combined_cache)

Expiration Time: January 26, 2025 at 03:41:57 PM
   Current Time: January 26, 2025 at 03:19:49 PM
00:22:07 until expiration


In [46]:
if len(freddie_cache.list()) > 0:
    freddie_cache.refresh
    freddie_cache.delete()

Deleting CachedContent : projects/1026793852137/locations/us-central1/cachedContents/2358342490416742400


In [47]:
if len(fannie_cache.list()) > 0:
    fannie_cache.refresh
    fannie_cache.delete()

Deleting CachedContent : projects/1026793852137/locations/us-central1/cachedContents/1596108253484285952


In [49]:
if len(combined_cache.list()) > 0:
    combined_cache.refresh
    combined_cache.delete()

Deleting CachedContent : projects/1026793852137/locations/us-central1/cachedContents/3511263995023589376
